## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [12]:
%load_ext line_profiler

In [13]:
# 
import numpy as np

A = np.random.randint(0, 1000, size=1_000_000)
A

array([893,  28, 530, ..., 727,   9, 819])

In [14]:
def f1(A):
    acc, cnt = 0, 0
    for ai in A:
        bi=ai+100
        acc+=bi
        cnt+=1
    return acc/cnt

In [15]:
def f2(A):
    acc = 0
    for ai in A:
        bi=ai+100
        acc+=bi
    return acc/len(A)

In [16]:
def f3(A):
    return sum(A)/len(A) + 100

In [17]:
def f4(A):
    return A.mean()+100

In [18]:
# проверка времени выполнения функции

# %timeit f1(A) 
# %timeit f2(A)
# %timeit f3(A)
# %timeit f4(A)


In [19]:
# %lprun -f f2 f2(A)

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [20]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

In [21]:
%timeit g(df)

647 ms ± 3.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%lprun -f g g(df)

Timer unit: 1e-06 s

Total time: 0.65246 s
File: <ipython-input-20-e94296ffa06f>
Function: g at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def g(df):
    10         1          4.0      4.0      0.0      letters = ['a', 'b', 'c', 'd', 'e']
    11         1          1.0      1.0      0.0      dfs = []
    12         6          5.0      0.8      0.0      for letter in letters:
    13         5     631582.0 126316.4     96.8          q = df[df['key']==letter]
    14         5         11.0      2.2      0.0          dfs.append(q)
    15         1      20857.0  20857.0      3.2      return pd.concat(dfs, axis=0)

In [23]:
def g_op(df):
    return df[df["key"].isin(["a", "b", "c", "d", "e"])]


In [24]:
%timeit g_op(df)

80.8 ms ± 7.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [25]:
import pandas as pd
recipes = pd.read_csv("./data/recipes_sample.csv", sep=",", parse_dates=['submitted'])
reviews = pd.read_csv("./data/reviews_sample.csv", sep=",", parse_dates=['date'])

In [26]:
def f1(reviews):
    """А) С использованием метода `DataFrame.iterrows` исходной таблицы"""
    acc = 0
    for _, row in reviews.iterrows():
        acc += row["rating"]
    return acc/len(reviews)


def f2(reviews):
    """Б) С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год"""
    acc, cnt = 0, 0
    for _, row in reviews.iterrows():
        if (pd.to_datetime(row["date"], format="%d.%m.%Y").year == 2010) == True:
            acc += row.loc["rating"]
            cnt += 1
    return acc/cnt


def f3(reviews):
    """В) С использованием метода `Series.mean`"""
    return reviews["rating"].mean()


# print(f1(reviews))
# print(f2(reviews))
# print(f3(reviews))


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [27]:
# %timeit f1(reviews)
# %lprun -f f1 f1(reviews)
"""11.2 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)"""

'11.2 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)'

In [28]:
# %timeit f2(reviews)
# %lprun -f f2 f2(reviews)
"""12 s ± 172 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)"""

'12 s ± 172 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)'

In [29]:
# %timeit f3(reviews)
# %lprun -f f3 f3(reviews)
"""168 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)"""


'168 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)'

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [30]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

# %lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-06 s

Total time: 79.166 s
File: <ipython-input-16-69ce61806718>
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count(df):
     2         1          2.0      2.0      0.0      word_reviews = {}
     3    126680   23382880.0    184.6     29.5      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679    7692559.0     60.7      9.7          recipe_id, review = row['recipe_id'], row['review']
     5    126679     643279.0      5.1      0.8          words = review.split(' ')
     6   6918689    2627727.0      0.4      3.3          for word in words:
     7   6792010    3726334.0      0.5      4.7              if word not in word_reviews:
     8    174944     110219.0      0.6      0.1                  word_reviews[word] = []
     9   6792010    3777497.0      0.6      4.8              word_reviews[word].append(recipe_id)
    10                                               
    11         1          2.0      2.0      0.0      word_reviews_count = {}
    12    126680   23701415.0    187.1     29.9      for _, row in df.dropna(subset=['review']).iterrows():
    13    126679    4451817.0     35.1      5.6          review = row['review']
    14    126679     652717.0      5.2      0.8          words = review.split(' ')
    15   6918689    2728613.0      0.4      3.4          for word in words:
    16   6792010    5670984.0      0.8      7.2              word_reviews_count[word] = len(word_reviews[word])
    17         1          1.0      1.0      0.0      return word_reviews_count


1) метод iterrows очень медленный, поэтому избавляемся от цикла, тем более что цикл итерируется по одному и тому же df два раза
2) кол-во подсчета слов также встречается два раза (word_reviews/word_reviews_count)

In [32]:
def upgraded_solution(df):
    word_reviews_count = {}
    
    for _, row in df.dropna(subset=["review"]).iterrows():
        word_list = row["review"].split(" ")  # список всех слов

        for i in word_list:
            if i not in word_reviews_count:  # если видим слово первый раз
                word_reviews_count[i] = 0

            word_reviews_count[i] += 1

    return word_reviews_count


# upgraded_solution(reviews)


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [33]:
# %lprun -f upgraded_solution upgraded_solution(reviews)

Timer unit: 1e-06 s

Total time: 38.0768 s
File: <ipython-input-32-5f273a519aca>
Function: upgraded_solution at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def upgraded_solution(df):
     2         1          2.0      2.0      0.0      word_reviews_count = {}
     3    126680   23409402.0    184.8     61.5      for _, row in df.dropna(subset=["review"]).iterrows():
     4    126679    5007192.0     39.5     13.2          word_list = row["review"].split(" ") # список всех слов
     5                                                   
     6   6918689    2486597.0      0.4      6.5          for i in word_list:
     7   6792010    3648808.0      0.5      9.6              if i not in word_reviews_count:
     8    174944      92715.0      0.5      0.2                  word_reviews_count[i] = 0
     9   6792010    3432085.0      0.5      9.0              word_reviews_count[i] += 1
    10                              

Timer unit: 1e-06 s

Total time: 38.0768 s
File: <ipython-input-32-5f273a519aca>
Function: upgraded_solution at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def upgraded_solution(df):
     2         1          2.0      2.0      0.0      word_reviews_count = {}
     3    126680   23409402.0    184.8     61.5      for _, row in df.dropna(subset=["review"]).iterrows():
     4    126679    5007192.0     39.5     13.2          word_list = row["review"].split(" ") # список всех слов
     5                                                   
     6   6918689    2486597.0      0.4      6.5          for i in word_list:
     7   6792010    3648808.0      0.5      9.6              if i not in word_reviews_count:
     8    174944      92715.0      0.5      0.2                  word_reviews_count[i] = 0
     9   6792010    3432085.0      0.5      9.0              word_reviews_count[i] += 1
    10                                               
    11                                           
    12                                           
    13         1          2.0      2.0      0.0      return word_reviews_count

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [61]:
# Замечание: удалите из выборки отзывы с нулевым рейтингом.
new_df = reviews[reviews["rating"] != 0]
# print(len(reviews), len(new_df))

In [71]:
import numpy as np
from numba import jit
def without_all(A_t, F_t):
    """1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`"""
    # A_t - actual value; F_t - forecast value
    # doesn`t work
    return sum([abs(A_t[i] - F_t[i]) / A_t[i] for i in A_t]) / len(A_t) * 100


def with_numpy_not_numba(A_t, F_t): 
    """3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`"""
    y_true, y_pred = np.array(A_t), np.array(F_t)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(without_all([1, 2, 3], [4, 6, 7]))
print(with_numpy_not_numba([1, 2, 3], [4, 6, 7]))


# Я не математик, сорян, а еще и numba не фурычит

IndexError: list index out of range